In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [3]:
NUM_ROUNDS = 10
NUM_CLIENTS = 10
BATCH_SIZE = 10
EPOCHS_PER_ROUND = 5
LEARNING_RATE = 0.1
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-7

In [4]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert into Categorical (One-Hot Encoding)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
global_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28, 28)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [6]:
global_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
x_train_clients = np.array_split(x_train,NUM_CLIENTS)
y_train_clients = np.array_split(y_train,NUM_CLIENTS)

x_test_clients = np.array_split(x_test,NUM_CLIENTS)
y_test_clients = np.array_split(y_test,NUM_CLIENTS)

In [8]:
client_models = []
for i in range(NUM_CLIENTS):
    local_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    local_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    local_model.fit(x_train_clients[i],y_train_clients[i],epochs=EPOCHS_PER_ROUND, batch_size=BATCH_SIZE,verbose=0)
    acc = local_model.evaluate(x_test_clients[i],y_test_clients[i])
    print(acc)
    client_models.append(local_model)

32/32 [==============================] - 0s 1ms/step - loss: 0.2182 - accuracy: 0.9290
[0.21819111704826355, 0.9290000200271606]
32/32 [==============================] - 0s 1ms/step - loss: 0.3068 - accuracy: 0.9180
[0.30677688121795654, 0.9179999828338623]
32/32 [==============================] - 0s 1ms/step - loss: 0.2764 - accuracy: 0.9310
[0.2763940393924713, 0.9309999942779541]
32/32 [==============================] - 0s 1ms/step - loss: 0.2937 - accuracy: 0.9240
[0.29369282722473145, 0.9240000247955322]
32/32 [==============================] - 0s 2ms/step - loss: 0.2860 - accuracy: 0.9140
[0.2860265076160431, 0.9139999747276306]
32/32 [==============================] - 0s 990us/step - loss: 0.1687 - accuracy: 0.9530
[0.16870750486850739, 0.953000009059906]
32/32 [==============================] - 0s 1ms/step - loss: 0.2272 - accuracy: 0.9430
[0.22724786400794983, 0.9430000185966492]
32/32 [==============================] - 0s 4ms/step - loss: 0.0990 - accuracy: 0.9700
[0.09897620

In [10]:
acc = global_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 0s 1ms/step - loss: 2.3934 - accuracy: 0.0969
[2.3933961391448975, 0.09690000116825104]


In [9]:
# SERVER ADAM OPTIMIZER
fedadam = tf.optimizers.Adam(learning_rate=LEARNING_RATE,beta_1=BETA_1,beta_2 = BETA_2,epsilon=EPSILON)

In [11]:
# Performing Federated Averaging
weights = global_model.get_weights()
for i in range(len(weights)):
    for j in range(NUM_CLIENTS):
        client_weights = client_models[j].get_weights()
        weights[i] += client_weights[i]/NUM_CLIENTS

global_model.set_weights(weights)

In [12]:
acc = global_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 0s 2ms/step - loss: 1.9549 - accuracy: 0.2610
[1.9548991918563843, 0.26100000739097595]


In [13]:
# Train the Fed AVG model
fed_model = global_model
fed_model.compile(optimizer=fedadam, loss='categorical_crossentropy',metrics=['accuracy'])
fed_model.fit(x_train,y_train,epochs=5,batch_size=20)

Epoch 1/5
3000/3000 [==============================] - 7s 2ms/step - loss: 1.4940 - accuracy: 0.5128
Epoch 2/5
3000/3000 [==============================] - 6s 2ms/step - loss: 1.5567 - accuracy: 0.4692
Epoch 3/5
3000/3000 [==============================] - 6s 2ms/step - loss: 1.5028 - accuracy: 0.4923
Epoch 4/5
3000/3000 [==============================] - 5s 2ms/step - loss: 1.4998 - accuracy: 0.4790
Epoch 5/5
3000/3000 [==============================] - 6s 2ms/step - loss: 1.4656 - accuracy: 0.5017


In [14]:
acc = fed_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 0s 1ms/step - loss: 1.7490 - accuracy: 0.5414
[1.7490007877349854, 0.5414000153541565]
